# Import libraries

In [1]:
import pandas as pd
import numpy as np
import mysql.connector

# Import conector
This didn't work due to problems with the connection with MySQL Server

# Importing Data and Creating DataFrames
# Duration Table

In [35]:
#import table
duration_df = pd.read_csv("/home/inrx/Ironhack/Projects/Project-Week-2-Lisbon/dataset-lx-transp/IV.16.csv")

In [36]:
# rename column
duration_df = duration_df.rename(columns={"MUN":"Motive", "motor_bike":"motorbike","tax":"taxi"})

In [37]:
# Slice row table for work
work_duration = duration_df[duration_df['Motive'] == 'Work']
work_duration.head()

,Motive,car_driver,car_pass,motorbike,taxi,public_bus,private_bus,train,metro,boat,plane,walk,bike,other
0,Work,24.8,25.5,19.5,22.6,48.3,33.2,51.0,41.4,54.2,262.7,15.1,32.2,56.2


In [38]:
# Slice row table for back home
back_home_duration = duration_df[duration_df['Motive'] =='Back home']

#back_home_duration.head()

In [39]:
# Append work and back home rows on the same table
work_duration = work_duration.append(back_home_duration)
#work_duration.head()

In [40]:
# checking for NaN 
work_duration.isnull().sum() 

Motive         0
car_driver     0
car_pass       0
motorbike      0
taxi           0
public_bus     0
private_bus    0
train          0
metro          0
boat           0
plane          0
walk           0
bike           0
other          0
dtype: int64

In [41]:
work_duration.describe()

,car_driver,car_pass,motorbike,taxi,public_bus,private_bus,train,metro,boat,plane,walk,bike,other
count,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000
mean,23.450000,23.30000,18.950000,20.850000,47.800000,35.200000,52.750000,42.350000,57.550000,249.600000,16.000000,35.90000,44.150000
std,1.909188,3.11127,0.777817,2.474874,0.707107,2.828427,2.474874,1.343503,4.737615,18.526198,1.272792,5.23259,17.041273
min,22.100000,21.10000,18.400000,19.100000,47.300000,33.200000,51.000000,41.400000,54.200000,236.500000,15.100000,32.20000,32.100000
25%,22.775000,22.20000,18.675000,19.975000,47.550000,34.200000,51.875000,41.875000,55.875000,243.050000,15.550000,34.05000,38.125000
50%,23.450000,23.30000,18.950000,20.850000,47.800000,35.200000,52.750000,42.350000,57.550000,249.600000,16.000000,35.90000,44.150000
75%,24.125000,24.40000,19.225000,21.725000,48.050000,36.200000,53.625000,42.825000,59.225000,256.150000,16.450000,37.75000,50.175000
max,24.800000,25.50000,19.500000,22.600000,48.300000,37.200000,54.500000,43.300000,60.900000,262.700000,16.900000,39.60000,56.200000


In [43]:
work_duration = work_duration.drop(columns=['other','taxi'])
work_duration

,Motive,car_driver,car_pass,motorbike,public_bus,private_bus,train,metro,boat,walk,bike
0,Work,24.8,25.5,19.5,48.3,33.2,51.0,41.4,54.2,15.1,32.2
7,Back home,22.1,21.1,18.4,47.3,37.2,54.5,43.3,60.9,16.9,39.6


# Distance Table

In [10]:
#import
distance_df = pd.read_csv ("/home/inrx/Ironhack/Projects/Project-Week-2-Lisbon/dataset-lx-transp/IV.17.csv")

distance_df.head()

,Motive,car_driver,car_pass,motorbike,táx,public_bus,private_bus,train,metro,boat,plaine,walk,bike,other
0,Work,14.7,15.8,13.3,9.4,12.5,23.4,18.5,8.9,19.0,906.0,1.4,8.0,38.8
1,Study,14.1,5.7,10.1,8.3,10.7,7.1,20.1,7.5,20.6,NaN,1.5,7.5,59.1
2,Accompany,7.5,10.0,7.6,4.7,8.6,9.4,16.0,7.1,18.4,NaN,1.3,0.9,13.3
3,Recreaton,19.0,20.7,13.2,6.6,19.0,33.1,13.6,5.7,16.1,519.1,2.0,9.2,46.1
4,Shopping,7.7,10.6,7.9,3.9,6.7,5.0,15.5,6.6,6.6,NaN,1.4,8.9,14.0


In [46]:
distance_df = distance_df.drop(columns=['plane','other','taxi'])
distance_df

,Motive,car_driver,car_pass,motorbike,public_bus,private_bus,train,metro,boat,walk,bike
0,Work,24.8,25.5,19.5,48.3,33.2,51.0,41.4,54.2,15.1,32.2
1,Study,27.9,13.8,17.4,39.9,22.9,53.6,38.2,54.0,17.2,24.4
2,Accompany,15.8,17.8,17.9,37.2,20.5,43.8,36.5,74.4,14.6,9.6
3,Recreation,26.5,27.8,17.2,46.9,41.1,52.5,31.7,56.3,23.3,41.0
4,Shopping,15.6,17.0,13.7,32.9,21.3,46.8,29.0,51.5,15.3,29.3
5,Personal matters,26.7,25.8,18.7,47.6,28.8,60.8,27.7,63.7,17.4,41.0
6,Other,24.0,24.5,8.3,48.8,24.0,58.6,48.4,NaN,16.2,33.2
7,Back home,22.1,21.1,18.4,47.3,37.2,54.5,43.3,60.9,16.9,39.6


In [48]:
work_distance = distance_df[distance_df['Motive'] == 'Work']
back_home_distance = distance_df[distance_df['Motive'] =='Back home']
work_distance = work_distance.append(back_home_distance)
work_distance.head()

,Motive,car_driver,car_pass,motorbike,public_bus,private_bus,train,metro,boat,walk,bike
0,Work,24.8,25.5,19.5,48.3,33.2,51.0,41.4,54.2,15.1,32.2
7,Back home,22.1,21.1,18.4,47.3,37.2,54.5,43.3,60.9,16.9,39.6


In [49]:
work_distance.describe()

,car_driver,car_pass,motorbike,public_bus,private_bus,train,metro,boat,walk,bike
count,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000
mean,23.450000,23.30000,18.950000,47.800000,35.200000,52.750000,42.350000,57.550000,16.000000,35.90000
std,1.909188,3.11127,0.777817,0.707107,2.828427,2.474874,1.343503,4.737615,1.272792,5.23259
min,22.100000,21.10000,18.400000,47.300000,33.200000,51.000000,41.400000,54.200000,15.100000,32.20000
25%,22.775000,22.20000,18.675000,47.550000,34.200000,51.875000,41.875000,55.875000,15.550000,34.05000
50%,23.450000,23.30000,18.950000,47.800000,35.200000,52.750000,42.350000,57.550000,16.000000,35.90000
75%,24.125000,24.40000,19.225000,48.050000,36.200000,53.625000,42.825000,59.225000,16.450000,37.75000
max,24.800000,25.50000,19.500000,48.300000,37.200000,54.500000,43.300000,60.900000,16.900000,39.60000


# Layover Table

In [227]:
layover_df = pd.read_csv ("/home/inrx/Ironhack/Projects/Project-Week-2-Lisbon/dataset-lx-transp/IV.18.csv", \
                          header=None, names=["transport","Total","0","1","> 2","a","b","c","d"])

layover_df.head(10)

,transport,Total,0,1,> 2,a,b,c,d
0,car_driver,2475191.73,2381648.35,88253.78,5289.61,NaN,NaN,NaN,NaN
1,car_pass,697820.74,671130.21,23897.16,2793.38,NaN,NaN,NaN,NaN
2,motorbike,47330.20,46737.23,578.20,14.77,NaN,NaN,NaN,NaN
3,taxi,19219.94,18223.16,958.53,38.26,NaN,NaN,NaN,NaN
4,public_bus,420837.65,232869.94,110466.85,77500.86,NaN,NaN,NaN,NaN
5,private_bus,51387.95,45063.97,4220.71,2103.28,NaN,NaN,NaN,NaN
6,Train,173668.77,30704.62,53929.71,89034.43,NaN,NaN,NaN,NaN
7,metro,166826.93,82242.73,60236.34,24347.85,NaN,NaN,NaN,NaN
8,boat,16564.01,1424.51,3002.83,12136.66,NaN,NaN,NaN,NaN
9,bike,26186.88,24259.93,1874.96,51.99,NaN,NaN,NaN,NaN


In [228]:
89034/73668

1.2085844600097735

In [229]:
layover_df.dtypes

transport     object
Total        float64
0            float64
1            float64
> 2          float64
a            float64
b            float64
c            float64
d            float64
dtype: object

In [230]:
layover_columns = ["a","b","c","d"]
layover_df = layover_df.drop(layover_columns, axis = 1)
layover_df

,transport,Total,0,1,> 2
0,car_driver,2475191.73,2381648.35,88253.78,5289.61
1,car_pass,697820.74,671130.21,23897.16,2793.38
2,motorbike,47330.20,46737.23,578.20,14.77
3,taxi,19219.94,18223.16,958.53,38.26
4,public_bus,420837.65,232869.94,110466.85,77500.86
5,private_bus,51387.95,45063.97,4220.71,2103.28
6,Train,173668.77,30704.62,53929.71,89034.43
7,metro,166826.93,82242.73,60236.34,24347.85
8,boat,16564.01,1424.51,3002.83,12136.66
9,bike,26186.88,24259.93,1874.96,51.99


In [231]:
layover_df["0"]=layover_df["0"]*100/layover_df["Total"] 
layover_df["1"]=layover_df["1"]*100/layover_df["Total"] 
layover_df["> 2"]=layover_df["> 2"]*100/layover_df["Total"]
layover_df["Total"]=layover_df["Total"]*100/layover_df["Total"]
layover_df.head(10)max

,transport,Total,0,1,> 2
0,car_driver,100.0,96.220762,3.565533,0.213705
1,car_pass,100.0,96.175160,3.424541,0.400301
2,motorbike,100.0,98.747164,1.221630,0.031206
3,taxi,100.0,94.813824,4.987164,0.199064
4,public_bus,100.0,55.334864,26.249279,18.415857
5,private_bus,100.0,87.693652,8.213424,4.092944
6,Train,100.0,17.679989,31.053200,51.266805
7,metro,100.0,49.298234,36.107084,14.594676
8,boat,100.0,8.600031,18.128642,73.271267
9,bike,100.0,92.641544,7.159921,0.198535


In [249]:
max_layovers = layover_df["> 2"].max()

# Find the mode of transportation with max layovers
max_layovers_row = layover_df.loc[layover_df['> 2'] == max_layovers].index.item()


## Then, given the index, we find the transport of the corresponding index
max_layovers_transport = layover_df.loc[max_layovers_row,'transport']

max_layovers_transport = max_layovers_transport.lower()

print (f'The mode of transport with more layovers is {max_layovers_transport}, \
with {round(max_layovers,2)} % of total people who commute by {max_layovers_transport} \
doing more than 2 layovers.')

The mode of transport with more layovers is boat, with 73.27 % of total people who commute by boat doing more than 2 layovers.


In [252]:
min_layovers = layover_df["0"].max()

# Find the mode of transportation with max layovers
min_layovers_row = layover_df.loc[layover_df['0'] == min_layovers].index.item()

## Then, given the index, we find the transport of the corresponding index
min_layovers_transport = layover_df.loc[min_layovers_row,'transport']

min_layovers_transport = min_layovers_transport.lower()

print (f'The mode of transport with fewer layovers is {min_layovers_transport}, \
with {round(min_layovers,2)} % of total people who commute by {min_layovers_transport} \
not doing layovers.')

The mode of transport with fewer layovers is motorbike, with 98.75 % of total people who commute by motorbike not doing layovers.


# Is there a correlation between commute duration and the number of layovers?


In [301]:
work_duration_t=work_duration.transpose()
work_duration_t

,0,7
Motive,Work,Back home
car_driver,24.8,22.1
car_pass,25.5,21.1
motorbike,19.5,18.4
public_bus,48.3,47.3
private_bus,33.2,37.2
train,51,54.5
metro,41.4,43.3
boat,54.2,60.9
walk,15.1,16.9


In [302]:
work_duration_t = work_duration_t.rename(columns=work_duration_t.iloc[0])

In [303]:
work_duration_t = work_duration_t.drop("Motive", axis=0)
work_duration_t

,Work,Back home
car_driver,24.8,22.1
car_pass,25.5,21.1
motorbike,19.5,18.4
public_bus,48.3,47.3
private_bus,33.2,37.2
train,51,54.5
metro,41.4,43.3
boat,54.2,60.9
walk,15.1,16.9
bike,32.2,39.6


In [322]:
work_duration_t_no_walk = work_duration_t.drop("walk", axis=0)
work_duration_t_no_walk

,Work,Back home
car_driver,24.8,22.1
car_pass,25.5,21.1
motorbike,19.5,18.4
public_bus,48.3,47.3
private_bus,33.2,37.2
train,51,54.5
metro,41.4,43.3
boat,54.2,60.9
bike,32.2,39.6


In [319]:
max_duration = work_duration_t["Work"].max()

# Find the mode of transportation with max duration
max_duration_row = work_duration_t.loc[work_duration_t['Work'] == max_duration].index.item()

#max_duration_transport = max_duration_transport.lower()

print (f'The mode of transport that takes longer is {max_duration_row}, \
consuming about {max_duration} min daily.')

54.2
boat
The mode of transport that takes longer is boat, consuming about 54.2 min daily


In [324]:
min_duration = work_duration_t_no_walk["Work"].min()

min_duration_row = work_duration_t_no_walk.loc[work_duration_t_no_walk['Work'] == min_duration].index.item()

print (f'The quickest mode of transport {min_duration_row}, \
consuming about {min_duration} min daily.')

The quickest mode of transport motorbike, consuming about 19.5 min daily.


In [325]:
if max_layovers_transport == max_duration_row and min_layovers_transport == min_duration_row:
    print (f'The number of layovers influences the commute duration')
    
else:
    print (f"there is no relationship between the number of layovers and commute duration")

The number of layovers influences the commute duration


# Calculating Speed

In [16]:
velocity_df = work_distance.copy()
velocity_df

,Motive,car_driver,car_pass,motorbike,taxi,public_bus,private_bus,train,metro,boat,plane,walk,bike,other
0,Work,24.8,25.5,19.5,22.6,48.3,33.2,51.0,41.4,54.2,262.7,15.1,32.2,56.2
7,Back home,22.1,21.1,18.4,19.1,47.3,37.2,54.5,43.3,60.9,236.5,16.9,39.6,32.1


# Calculations

In [70]:
layover_df["0"].max()

'82,243'

In [71]:
layover_df["1"].max()

'959'